In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
#import random
#import sklearn
#import collections
#from sklearn.model_selection import train_test_split
#import json
#import pylab 
#from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import matplotlib
import matplotlib.patches as mpatches
#import shap
#import pandas as pd
import tensorflow as tf
#import tarfile
from tensorflow.keras.models import load_model
#from qkeras import QActivation, QDense, QConv2D, QBatchNormalization
import ensembler_functions as ef
import tf2onnx
import onnx
import os
from sklearn.model_selection import train_test_split
import load_and_match as lam

2024-10-30 20:35:15.360753: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 20:35:15.412256: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Welcome to JupyROOT 6.30/04


In [2]:
# Set matplotlib default color cycle
new_color_cycle = [
    '#1f77b4',
    '#ff7f0e',
    '#2ca02c',
    '#d62728',
    '#9467bd',
    '#8c564b',
    '#e377c2',
    '#7f7f7f',
    '#bcbd22',
    '#17becf',
    '#aec7e8',
    '#ffbb78',
    '#98df8a',
    '#ff9896',
    '#c5b0d5',
    '#c49c94',
    '#f7b6d2',
    '#c7c7c7',
    '#dbdb8d',
    '#9edae5'
]

# You can then apply this new color cycle to your matplotlib plots
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=new_color_cycle)

In [3]:
L1AD_rate = 1000
target_rate = 10

In [ ]:
# Load and match my data with the topo2A (L1AD) scores. Only needs to be done once ever.
lam.load_and_match('./h5_ntuples/11-5-2024')

In [ ]:
data_info = {
    "train_data_scheme": "topo2A_train+overlap", 
    "pt_normalization_type": "StandardScaler", 
    "L1AD_rate": 1000
}

training_info = {
    "save_path": "./trained_models/multiple_trainings/trial_8", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": True, 
    "num_trainings": 10,
    "training_weights": True
}

datasets, data_info = ef.load_and_preprocess(**data_info)
training_info, data_info = ef.train_multiple_models(datasets, data_info, **training_info)

In [ ]:
ef.process_multiple_models(
    training_info=training_info,
    data_info=data_info,
    plots_path=training_info['save_path']+'/plots',
    target_rate=target_rate,
    L1AD_rate=L1AD_rate
)

### Test ONNX

In [7]:
training_info = {
    "save_path": "./trained_models/multiple_trainings/trial_2", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": True, 
    "num_trainings": 10,
    "training_weights": True
}

data_info = {
    "train_data_scheme": 
    "topo2A_train+overlap", 
    "pt_normalization_type": 
    "global_division", 
    "L1AD_rate": 1000
}
model_version=0
ef.convert_to_onnx(training_info=training_info, model_version=model_version, object_type='HLT', save_dir='./trained_models/multiple_trainings/trial_2/onnx')
ef.convert_to_onnx(training_info=training_info, model_version=model_version, object_type='L1', save_dir='./trained_models/multiple_trainings/trial_2/onnx')
datasets, data_info = ef.load_and_preprocess(**data_info)
datasets = ef.compare_tf_with_onnx(datasets=datasets, training_info=training_info, model_version=0, onnx_path='./trained_models/multiple_trainings/trial_2/onnx')

Could not search for non-variable resources. Concrete function internal representation may have changed.
2024-10-30 15:45:26.715362: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-10-30 15:45:26.715559: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2024-10-30 15:45:27.230193: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-10-30 15:45:27.230453: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session


ONNX HLT_AE model saved to: ./trained_models/multiple_trainings/trial_2/onnx/HLT_AE_0.onnx


Could not search for non-variable resources. Concrete function internal representation may have changed.
2024-10-30 15:45:29.812788: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-10-30 15:45:29.812992: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2024-10-30 15:45:30.345443: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-10-30 15:45:30.345661: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session


ONNX L1_AE model saved to: ./trained_models/multiple_trainings/trial_2/onnx/L1_AE_0.onnx
Loaded A14N23LO from ./h5_ntuples/A14N23LO.h5
Loaded EB from ./h5_ntuples/EB.h5
Loaded EB_test2 from ./h5_ntuples/EB_test2.h5
Loaded EB_train from ./h5_ntuples/EB_train.h5
Loaded HAHMggfZdZd2l2nu from ./h5_ntuples/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from ./h5_ntuples/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from ./h5_ntuples/HLT_noalg_eb_L1All.h5
Loaded ZZ4lep from ./h5_ntuples/ZZ4lep.h5
Loaded Zprime2EJs from ./h5_ntuples/Zprime2EJs.h5
Loaded jjJZ1 from ./h5_ntuples/jjJZ1.h5
Loaded jjJZ2 from ./h5_ntuples/jjJZ2.h5
Loaded jjJZ4 from ./h5_ntuples/jjJZ4.h5
Loaded qqa from ./h5_ntuples/qqa.h5


In [8]:
for tag, data_dict in datasets.items():
    print(f'{tag}:')
    for key, value in data_dict.items():
        print(f'    {key}: {value.shape}')

A14N23LO:
    HLT_data: (10000, 48)
    L1_data: (10000, 48)
    passHLT: (10000,)
    passL1: (10000,)
    topo2A_AD_scores: (10000,)
    weights: (10000,)
    L1Seeded: (10000,)
    HLT_model_outputs: (10000, 48)
    L1_model_outputs: (10000, 48)
    HLT_AD_scores: (10000,)
    L1_AD_scores: (10000,)
    ONNX_HLT_model_outputs: (10000, 48)
    ONNX_L1_model_outputs: (10000, 48)
    ONNX_HLT_AD_scores: (10000,)
    ONNX_L1_AD_scores: (10000,)
HAHMggfZdZd2l2nu:
    HLT_data: (70000, 48)
    L1_data: (70000, 48)
    passHLT: (70000,)
    passL1: (70000,)
    topo2A_AD_scores: (70000,)
    weights: (70000,)
    L1Seeded: (70000,)
    HLT_model_outputs: (70000, 48)
    L1_model_outputs: (70000, 48)
    HLT_AD_scores: (70000,)
    L1_AD_scores: (70000,)
    ONNX_HLT_model_outputs: (70000, 48)
    ONNX_L1_model_outputs: (70000, 48)
    ONNX_HLT_AD_scores: (70000,)
    ONNX_L1_AD_scores: (70000,)
HHbbttHadHad:
    HLT_data: (100000, 48)
    L1_data: (100000, 48)
    passHLT: (100000,)
    pa

In [9]:
print(datasets['EB_test']['HLT_AD_scores'][0:5])
print()
print(datasets['EB_test']['ONNX_HLT_AD_scores'][0:5])

[1.62544931 1.92602879 1.33561259 0.         1.06108748]

[1.62544931 1.92602879 1.33561259 0.         1.06108748]


In [ ]:
# Success! These AD scores are the same.